In [27]:
# This is Derived from 
# https://github.com/gkamradt/langchain-tutorials/blob/main/data_generation/5%20Levels%20Of%20Summarization%20-%20Novice%20To%20Expert.ipynb

# Workaround OpenAI's Token Limit
# https://www.youtube.com/watch?v=f9_BWhCI4Zo&t=418s

In [2]:
!pip install langchain
!pip install python-dotenv


In [7]:

%load_ext dotenv
%dotenv


The dotenv extension is already loaded. To reload it, use:
  %reload_ext dotenv


In [35]:
%env OPENAI_KEY=''

env: OPENAI_KEY=''


In [3]:
import os
from dotenv import load_dotenv
from langchain import OpenAI
from langchain import PromptTemplate
from langchain.chains.summarize import load_summarize_chain
from langchain.text_splitter import RecursiveCharacterTextSplitter


In [ ]:
import os 
open_api_key = os.environ["OPENAI_KEY"]
print(open_api_key)



In [5]:
llm = OpenAI( temperature=0, openai_api_key= '')

In [13]:
# Level 0 try
prompt = """
Please provide a summary of the following text

TEXT:
Philosophy (from Greek: φιλοσοφία, philosophia, 'love of wisdom') \
is the systematized study of general and fundamental questions, \
such as those about existence, reason, knowledge, values, mind, and language. \
Some sources claim the term was coined by Pythagoras (c. 570 – c. 495 BCE), \
although this theory is disputed by some. Philosophical methods include questioning, \
critical discussion, rational argument, and systematic presentation.
"""
num_tokens = llm.get_num_tokens(prompt)
print( num_tokens)

121


In [23]:
output = llm(prompt)
print(output)


Philosophy is a systematized study of general and fundamental questions about existence, reason, knowledge, values, mind, and language. It is believed to have been coined by Pythagoras, and its methods include questioning, critical discussion, rational argument, and systematic presentation.


In [7]:
#This is Level 1 - Read large data
with open('data/IndieHacker-283.txt', 'r') as file: 
    file_contents = file.read()
    print(file_contents[:300])

it's up dude you know what's going on man who do we got today Liz and Lucas Herman I'm excited about the Hermits
 you know what I'm excited about Luke and his wife Liz they're sort of a team running this this company together
 their app I think they make $8,000 a month that was back in January so ma


In [8]:
llm.get_num_tokens(file_contents)

11314

In [10]:
# 1 token = 4 characters
# 1000 tokens = 4 *  1000 
# https://platform.openai.com/docs/models/gpt-4
text_splitter = RecursiveCharacterTextSplitter( separators=["\n\n", "\n", " "], chunk_size=4000, chunk_overlap=500)
docs = text_splitter.create_documents( [ file_contents])

In [16]:
num_docs = len(docs)
num_tokens_first_doc = llm.get_num_tokens(docs[0].page_content)
print(" num docs" , num_docs)
print(" num tokens ", num_tokens_first_doc)


 num docs 16
 num tokens  615


In [12]:
summary_chain = load_summarize_chain( llm=llm, chain_type='map_reduce')
output = summary_chain.run(docs)
print(output)

 Lucas and his wife Liz have created a successful app called Stage Timer.io which makes them more money than their startup jobs. They have used Reddit and Twitter to market their product and have gained the attention of Sam Perry from My First Million. They have also discussed the importance of having prior experience working together before starting a business, and the idea of having a family business. They have been successful in their partnership and have been able to use the money they make to pursue other interests. They have also discussed the success of entrepreneurs such as Robert Murdoch, Khan, Edison, and DaVinci, and the importance of being curious and having no succession planning.


In [13]:
map_prompt = """ 
Write a concise summary of following:
"{text}"
CONCISE SUMMARY:
"""
map_prompt_template = PromptTemplate( template=map_prompt, input_variables=["text"])

In [14]:
combine_prompt = """ 

Write a concise summary of text delimited by triple backquotes.
Return your response in bullet points which covers the key points of the text
```{text}```
BULLET POINT SUMMARY: 
"""
combine_prompt_template = PromptTemplate ( template= combine_prompt, input_variables=["text"])

In [15]:
summary_chain = load_summarize_chain( llm=llm , chain_type='map_reduce', map_prompt=map_prompt_template, combine_prompt=combine_prompt_template)

In [18]:
output = summary_chain.run(docs)

In [19]:
print(output)

- Lucas and his wife Liz have created a successful app that makes them $8,000 a month and an AI journalist bot
- They have an ambitious goal and have mapped out the steps they will take to reach it
- They have created an app called Stage Timer.io which has been successful and is now charging $20-30 a month for its services
- They have explored other industries and have built some tools
- They have found success in solving other people's problems and have even received their first paying customer
- They have grown their platform mainly through word of mouth and videos made by users
- They have used SEO and technical blogs and documentation to bring in paying customers
- They have a successful working relationship and have common interests
- They are determined to become rich and are taking steps to do so
- They have been experimenting with different business strategies and marketing strategies
- They have been able to make a comfortable living with their business and have been able to p